In [76]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os



訓練データとテストデータの読み込み

In [77]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

In [78]:
train

,index,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,native-country,Y
0,0,22,Private,132618,Some-college,12,Never-married,Tech-support,Own-child,White,Female,United-States,0
1,1,22,Private,132655,Bachelors,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,Mexico,1
2,2,23,?,132674,HS-grad,8,Never-married,Other-service,Not-in-family,White,Female,United-States,0
3,3,36,Private,132642,Bachelors,13,Married-civ-spouse,Sales,Husband,White,Male,United-States,1
4,4,49,Private,132646,Assoc-voc,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10195,10195,31,Private,132650,Bachelors,9,Divorced,Craft-repair,Other-relative,White,Female,Mexico,0
10196,10196,36,Private,132590,HS-grad,9,Married-civ-spouse,Exec-managerial,Unmarried,White,Male,Mexico,0
10197,10197,36,Local-gov,132498,Bachelors,13,Married-civ-spouse,?,Husband,Black,Male,United-States,0
10198,10198,17,Private,132614,Some-college,9,Never-married,?,Not-in-family,White,Female,United-States,0


In [79]:
test

,id,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,native-country
0,10200,30,Private,132646,HS-grad,12,Divorced,Prof-specialty,Unmarried,White,Male,Mexico
1,10201,30,Private,132626,HS-grad,4,Never-married,Sales,Wife,White,Female,United-States
2,10202,36,Private,132505,Bachelors,8,Married-civ-spouse,Prof-specialty,Husband,White,Male,Mexico
3,10203,23,Private,132610,HS-grad,9,Divorced,?,Not-in-family,White,Female,United-States
4,10204,36,Private,132468,12th,9,Married-civ-spouse,Sales,Husband,White,Male,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...
6795,16995,31,Private,132530,Some-college,9,Divorced,Handlers-cleaners,Not-in-family,White,Female,United-States
6796,16996,31,Private,132634,Bachelors,6,Never-married,Transport-moving,Not-in-family,Asian-Pac-Islander,Male,United-States
6797,16997,31,Private,132598,HS-grad,13,Widowed,Farming-fishing,Not-in-family,White,Male,United-States
6798,16998,23,Private,132573,Some-college,8,Divorced,?,Not-in-family,Black,Female,United-States


In [80]:
#index列・education列をドロップさせる
train = train.drop(['index', 'education'], axis=1)

#id列・education列をドロップさせる
test = test.drop(['id', 'education'], axis=1)


In [81]:
train

,age,workclass,fnlwgt,education-num,marital-status,occupation,relationship,race,sex,native-country,Y
0,22,Private,132618,12,Never-married,Tech-support,Own-child,White,Female,United-States,0
1,22,Private,132655,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,Mexico,1
2,23,?,132674,8,Never-married,Other-service,Not-in-family,White,Female,United-States,0
3,36,Private,132642,13,Married-civ-spouse,Sales,Husband,White,Male,United-States,1
4,49,Private,132646,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,1
...,...,...,...,...,...,...,...,...,...,...,...
10195,31,Private,132650,9,Divorced,Craft-repair,Other-relative,White,Female,Mexico,0
10196,36,Private,132590,9,Married-civ-spouse,Exec-managerial,Unmarried,White,Male,Mexico,0
10197,36,Local-gov,132498,13,Married-civ-spouse,?,Husband,Black,Male,United-States,0
10198,17,Private,132614,9,Never-married,?,Not-in-family,White,Female,United-States,0


In [82]:
test

,age,workclass,fnlwgt,education-num,marital-status,occupation,relationship,race,sex,native-country
0,30,Private,132646,12,Divorced,Prof-specialty,Unmarried,White,Male,Mexico
1,30,Private,132626,4,Never-married,Sales,Wife,White,Female,United-States
2,36,Private,132505,8,Married-civ-spouse,Prof-specialty,Husband,White,Male,Mexico
3,23,Private,132610,9,Divorced,?,Not-in-family,White,Female,United-States
4,36,Private,132468,9,Married-civ-spouse,Sales,Husband,White,Male,United-States
...,...,...,...,...,...,...,...,...,...,...
6795,31,Private,132530,9,Divorced,Handlers-cleaners,Not-in-family,White,Female,United-States
6796,31,Private,132634,6,Never-married,Transport-moving,Not-in-family,Asian-Pac-Islander,Male,United-States
6797,31,Private,132598,13,Widowed,Farming-fishing,Not-in-family,White,Male,United-States
6798,23,Private,132573,8,Divorced,?,Not-in-family,Black,Female,United-States


In [83]:
#欠損値がないか確認
print(train.isnull().sum())
print(test.isnull().sum())

age               0
workclass         0
fnlwgt            0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
native-country    0
Y                 0
dtype: int64
age               0
workclass         0
fnlwgt            0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
native-country    0
dtype: int64


In [84]:
#?の数を確認
print(train.isin(['?']).sum())
print(test.isin(['?']).sum())

age                 0
workclass         695
fnlwgt              0
education-num       0
marital-status      0
occupation        832
relationship        0
race                0
sex                 0
native-country      0
Y                   0
dtype: int64
age                 0
workclass         450
fnlwgt              0
education-num       0
marital-status      0
occupation        556
relationship        0
race                0
sex                 0
native-country      0
dtype: int64


In [85]:
train = train.replace('?', np.NaN)
test = test.replace('?', np.NaN)

In [86]:
train

,age,workclass,fnlwgt,education-num,marital-status,occupation,relationship,race,sex,native-country,Y
0,22,Private,132618,12,Never-married,Tech-support,Own-child,White,Female,United-States,0
1,22,Private,132655,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,Mexico,1
2,23,NaN,132674,8,Never-married,Other-service,Not-in-family,White,Female,United-States,0
3,36,Private,132642,13,Married-civ-spouse,Sales,Husband,White,Male,United-States,1
4,49,Private,132646,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,1
...,...,...,...,...,...,...,...,...,...,...,...
10195,31,Private,132650,9,Divorced,Craft-repair,Other-relative,White,Female,Mexico,0
10196,36,Private,132590,9,Married-civ-spouse,Exec-managerial,Unmarried,White,Male,Mexico,0
10197,36,Local-gov,132498,13,Married-civ-spouse,NaN,Husband,Black,Male,United-States,0
10198,17,Private,132614,9,Never-married,NaN,Not-in-family,White,Female,United-States,0


In [87]:
test

,age,workclass,fnlwgt,education-num,marital-status,occupation,relationship,race,sex,native-country
0,30,Private,132646,12,Divorced,Prof-specialty,Unmarried,White,Male,Mexico
1,30,Private,132626,4,Never-married,Sales,Wife,White,Female,United-States
2,36,Private,132505,8,Married-civ-spouse,Prof-specialty,Husband,White,Male,Mexico
3,23,Private,132610,9,Divorced,NaN,Not-in-family,White,Female,United-States
4,36,Private,132468,9,Married-civ-spouse,Sales,Husband,White,Male,United-States
...,...,...,...,...,...,...,...,...,...,...
6795,31,Private,132530,9,Divorced,Handlers-cleaners,Not-in-family,White,Female,United-States
6796,31,Private,132634,6,Never-married,Transport-moving,Not-in-family,Asian-Pac-Islander,Male,United-States
6797,31,Private,132598,13,Widowed,Farming-fishing,Not-in-family,White,Male,United-States
6798,23,Private,132573,8,Divorced,NaN,Not-in-family,Black,Female,United-States


In [88]:
#データ型を確認
print(train.dtypes)

age                int64
workclass         object
fnlwgt             int64
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
native-country    object
Y                  int64
dtype: object


In [89]:
categorical_features = [
    'workclass',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native-country']

In [90]:
def get_uniques(df, columns):
    uniques = dict()
    for column in columns:
        uniques[column] = list(df[column].unique())
    return uniques


In [91]:
get_uniques(train, categorical_features)

{'workclass': ['Private',
  nan,
  'Local-gov',
  'Self-emp-inc',
  'Federal-gov',
  'Self-emp-not-inc',
  'State-gov'],
 'marital-status': ['Never-married',
  'Married-civ-spouse',
  'Divorced',
  'Separated',
  'Widowed'],
 'occupation': ['Tech-support',
  'Farming-fishing',
  'Other-service',
  'Sales',
  'Exec-managerial',
  'Protective-serv',
  'Adm-clerical',
  'Prof-specialty',
  'Handlers-cleaners',
  'Craft-repair',
  nan,
  'Machine-op-inspct',
  'Transport-moving'],
 'relationship': ['Own-child',
  'Husband',
  'Not-in-family',
  'Unmarried',
  'Wife',
  'Other-relative'],
 'race': ['White', 'Asian-Pac-Islander', 'Black'],
 'sex': ['Female', 'Male'],
 'native-country': ['United-States', 'Mexico', 'Philippines']}

In [92]:
#ラベルエンコーディングする項目とOneHot encodingする項目を分割
label_encoding_features = ['sex']
one_hot_encoding_features = [
    'workclass', 
    'marital-status', 
    'occupation', 
    'relationship', 
    'race', 
    'native-country',
]


In [93]:
from sklearn import preprocessing
def label_encoder(df, columns):
    label_encoder = preprocessing.LabelEncoder()
    for column in columns:
        df[column] = label_encoder.fit_transform(df[column])
    return df


def onehot_encoder(df, columns):
    for column in columns:
        dummies = pd.get_dummies(df[column])
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    return df


In [94]:
train = label_encoder(train, label_encoding_features)
train = onehot_encoder(train, one_hot_encoding_features)

test = label_encoder(test, label_encoding_features)
test = onehot_encoder(test, one_hot_encoding_features)


In [95]:
train


,age,fnlwgt,education-num,sex,Y,Federal-gov,Local-gov,Private,Self-emp-inc,Self-emp-not-inc,...,Other-relative,Own-child,Unmarried,Wife,Asian-Pac-Islander,Black,White,Mexico,Philippines,United-States
0,22,132618,12,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,1
1,22,132655,9,1,1,0,0,1,0,0,...,0,0,0,0,0,0,1,1,0,0
2,23,132674,8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,36,132642,13,1,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
4,49,132646,13,1,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10195,31,132650,9,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,1,0,0
10196,36,132590,9,1,0,0,0,1,0,0,...,0,0,1,0,0,0,1,1,0,0
10197,36,132498,13,1,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
10198,17,132614,9,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1


In [96]:
test


,age,fnlwgt,education-num,sex,Federal-gov,Local-gov,Private,Self-emp-inc,Self-emp-not-inc,State-gov,...,Other-relative,Own-child,Unmarried,Wife,Asian-Pac-Islander,Black,White,Mexico,Philippines,United-States
0,30,132646,12,1,0,0,1,0,0,0,...,0,0,1,0,0,0,1,1,0,0
1,30,132626,4,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,1
2,36,132505,8,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,0,0
3,23,132610,9,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,36,132468,9,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6795,31,132530,9,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
6796,31,132634,6,1,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
6797,31,132598,13,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
6798,23,132573,8,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [97]:
#クロスバリデーションで評価
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True, random_state=1000)
for train_indices, val_indices in kf.split(train):
  print("train_indices", train_indices)
  print("val_indices", val_indices)



train_indices [    0     2     3 ... 10197 10198 10199]
val_indices [    1     6    14 ... 10187 10189 10193]
train_indices [    1     2     4 ... 10197 10198 10199]
val_indices [    0     3    17 ... 10190 10194 10195]
train_indices [    0     1     2 ... 10197 10198 10199]
val_indices [    5     7     9 ... 10185 10191 10196]
train_indices [    0     1     2 ... 10195 10196 10199]
val_indices [    4     8    12 ... 10192 10197 10198]
train_indices [    0     1     3 ... 10196 10197 10198]
val_indices [    2    11    19 ... 10178 10183 10199]


In [ ]:
#列名と通し番号の対応を辞書として管理
namemap = {}
for i, column in enumerate(train.columns):
  if column == 'Y':   #正解列はそのままとする
    namemap[column] = 'Y'
  else:               #それ以外は通し番号に変更
    namemap[column] = str(i)
